# Checking lab data

In [ ]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

sys.path.insert(0, '../../')
from brickstudy import foundation

In [ ]:
lab_data = pd.read_csv('../../secret_data/BRICK_lab_data_update_18_03_2024_2.csv', sep= ';', encoding='latin1')


In [ ]:

lab_data

In [ ]:
#pd.read_csv?

In [ ]:
lab_data2 = pd.read_excel('../../secret_data/Brick_lab_20230914.xlsx')

In [ ]:
lab_data2

In [ ]:
lab_data2.groupby('studieID').code_code_original

In [ ]:
lab_data2.groupby?